In [7]:
import os
import requests
import pandas as pd
import tarfile
import shutil
import glob

In [8]:
url = 'https://scontent-gmp1-1.xx.fbcdn.net/m1/v/t6/An8MNcSV8eixKBYJ2kyw6sfPh-J9U4tH2BV7uPzibNa0pu4uHi6fyXdlbADVO4nfvsWpTwR8B0usCARHTz33cBQNrC0kWZsD1MbBWjw.txt?ccb=10-5&oh=00_AfCqxWIpzDzhO1TM0tZ6Q_gBKoJTtyoNnW-GiDtUUqBNMg&oe=64B080D8&_nc_sid=f5a210'
df = pd.read_csv(url, sep='\t')  # Use sep parameter to specify the delimiter, if it's not a comma

In [9]:
FILE_NAMES = df['file_name'].tolist()
CDN_LINKS = df['cdn_link'].tolist()
#keep the txt file
ID_TXT =    [x for x in FILE_NAMES if '.txt' in x]
#save the link of its index
ID_TXT_LINK = [CDN_LINKS[FILE_NAMES.index(x)] for x in ID_TXT]
#remove this link from the list
CDN_LINKS = [x for x in CDN_LINKS if x not in ID_TXT_LINK]

# remove the entry with .txt
FILE_NAMES = [x for x in FILE_NAMES if '.txt' not in x]

#load image IDS in df from this ID_TXT_LINK 
#df_ids = pd.read_csv(ID_TXT_LINK[0], sep='\t')  # Use sep parameter to specify the delimiter, if it's not a comma

In [10]:
#TRAIN_FOLDER_INDICES = [300, 400, 500, 600, 700, 800, 900]#brain 0, 40,80, 120,160, 200, 
TRAIN_FOLDER_INDICES = [240, 280, 320, 360, 400, 440, 480, 520, 560, 600, 640, 680, 720, 760]
CDN_TRAIN_LINKS = [CDN_LINKS[i] for i in TRAIN_FOLDER_INDICES]

TEST_FOLDER_INDICES = [999]
CDN_TEST_LINKS = [CDN_LINKS[i] for i in TEST_FOLDER_INDICES]

In [11]:
# Set your target directory
n = 50  # Number of images to keep
DIR = 'data/train2/images'
DIR_GT = 'data/train2/ground_truth'

# Make sure the target directory exists
os.makedirs(DIR, exist_ok=True)
os.makedirs(DIR_GT, exist_ok=True)


for i, url in enumerate(CDN_TRAIN_LINKS):
    local_filename = f'file_{i}.tar'
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
               f.write(chunk)
    
    # Extract the tar file
    with tarfile.open(local_filename) as tar:
        tar.extractall(path=f'extracted_{i}')

    # Get list of all images (assuming they are jpg)
    images = glob.glob(f'extracted_{i}/*.jpg')
    gt = glob.glob(f'extracted_{i}/*.json')


    # Move the remaining images to the target directory and replace the old ones
    for idx, image in enumerate(images):
        if idx == n:
           break
        
        #shutil.move(image, DIR, copy_function=shutil.copy)
        #shutil.move(image.replace('.jpg', '.json'), DIR_GT)
        shutil.copy2(image, DIR)
        shutil.copy2(image.replace('.jpg', '.json'), DIR_GT)
        os.remove(image)
        os.remove(image.replace('.jpg', '.json'))
    shutil.rmtree(f'extracted_{i}')
    os.remove(local_filename)

KeyboardInterrupt: 

In [ ]:
n = 50  # Number of images to keep

# Set your target directory
DIR = 'data/val/images'
DIR_GT = 'data/val/ground_truth'

# Make sure the target directory exists
os.makedirs(DIR, exist_ok=True)
os.makedirs(DIR_GT, exist_ok=True)


for i, url in enumerate(CDN_TEST_LINKS):
    local_filename = f'file_{i}.tar'
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
               f.write(chunk)
    
    # Extract the tar file
    with tarfile.open(local_filename) as tar:
        tar.extractall(path=f'extracted_{i}')

    # Get list of all images (assuming they are jpg)
    images = glob.glob(f'extracted_{i}/*.jpg')
    gt = glob.glob(f'extracted_{i}/*.json')


    # Move the remaining images to the target directory and replace the old ones
    for idx, image in enumerate(images):
        if idx == n:
            break
        
        shutil.move(image, DIR, copy_function=shutil.copy)
        shutil.move(image.replace('.jpg', '.json'), DIR_GT)
        
        

    shutil.rmtree(f'extracted_{i}')
    os.remove(local_filename)

  

